In [ ]:
import serde.csv, pandas as pd
from pathlib import Path
from tum.actors.entry import *
from sm.namespaces.utils import KGName
from sm.dataset import Dataset
from tum.config import CRITICAL_MAAS_DIR

### Generate candidate graph

In [ ]:
actor = G.create_actor(
    MinmodGraphInferenceActor,
    [
        DBActorArgs(
            kgdbs=[
                KGDBArgs(
                    name=KGName.Generic,
                    version="20231130",
                    datadir=CRITICAL_MAAS_DIR / "data/databases",
                )
            ]
        ),
        DataActorArgs(skip_unk_ont_ent=True, skip_no_sm=True),
        MinmodGraphGenerationActorArgs(
            train_dsquery="darpa-criticalmaas",
            meta_prop_file=CRITICAL_MAAS_DIR
            / "ta2-table-understanding/data/meta_property/data.csv",
            top_n_stypes=2,
        ),
        MinmodGraphInferenceActorArgs(),
    ],
)

In [ ]:
test_exs = Dataset(CRITICAL_MAAS_DIR / "ta2-table-understanding/examples").load()

In [ ]:
with actor.data_actor.use_examples(
    "darpa-criticalmaas",
    Dataset(CRITICAL_MAAS_DIR / "ta2-table-understanding/data/known_models").load(),
    prefix="",
):
    output = []
    for test_ex in test_exs:
        with actor.data_actor.use_example(
            test_ex.id,
            test_ex,
        ) as testquery:
            sm = actor(testquery)
            output.append(sm[0])

In [ ]:
for sm in output:
    sm.draw()